In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from tqdm.auto import tqdm
from underthesea import sent_tokenize  


BASE_DIR = Path.cwd()        
DATA_DIR = BASE_DIR / "dataset"

CLEAN_PATH      = DATA_DIR / "clean_data.csv"

print("BASE_DIR :", BASE_DIR)
print("CLEAN   :", CLEAN_PATH, "->", CLEAN_PATH.exists())


c:\Users\kyen\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BASE_DIR : d:\do-an-tot-nghiep
CLEAN   : d:\do-an-tot-nghiep\dataset\clean_data.csv -> True


Đọc clean_data & xem nhanh

In [3]:
df = pd.read_csv(CLEAN_PATH)
print("Số dòng clean_data:", len(df))
print("Các cột:", df.columns.tolist())

# Kiểm tra lại phân bố subject
if "subject" in df.columns:
    print("\nPHÂN PHỐI SUBJECT:")
    counts = df["subject"].value_counts()
    for subj, c in counts.items():
        print(f"  {subj:25s}: {c:5d} mẫu ({c/len(df)*100:5.2f}%)")

df.head()


Số dòng clean_data: 11383
Các cột: ['title', 'lead', 'body', 'url', 'subject', 'published_at', 'source', 'original_subject']

PHÂN PHỐI SUBJECT:
  Sức khỏe                 :  1096 mẫu ( 9.63%)
  Kinh doanh               :  1094 mẫu ( 9.61%)
  Pháp luật                :  1062 mẫu ( 9.33%)
  Giải trí                 :  1049 mẫu ( 9.22%)
  Thể thao                 :  1029 mẫu ( 9.04%)
  Đời sống                 :  1029 mẫu ( 9.04%)
  Chính trị                :  1020 mẫu ( 8.96%)
  Du lịch                  :  1011 mẫu ( 8.88%)
  Khoa học công nghệ       :  1008 mẫu ( 8.86%)
  Thế giới                 :  1006 mẫu ( 8.84%)
  Giáo dục                 :   979 mẫu ( 8.60%)


,title,lead,body,url,subject,published_at,source,original_subject
0,Tổng Bí thư: Chọn nhân sự Bộ Chính trị khóa 14...,Tổng Bí thư Tô Lâm nhấn mạnh nhân sự Bộ Chính ...,"Sáng 5/11, phát biểu khai mạc Hội nghị Trung ư...",https://vnexpress.net/tong-bi-thu-chon-nhan-su...,Chính trị,2025-11-05T09:54:00-07:00,vnexpress,Chính trị
1,Khai mạc Hội nghị Trung ương 14,Hội nghị lần thứ 14 Ban Chấp hành Trung ương k...,"Tại phiên khai mạc, Tổng Bí thư Tô Lâm chủ trì...",https://vnexpress.net/khai-mac-hoi-nghi-trung-...,Chính trị,2025-11-05T09:02:00-07:00,vnexpress,Chính trị
2,Tổng Bí thư: Không để người dân phải xin những...,Tổng Bí thư Tô Lâm nhấn mạnh Nhà nước pháp quy...,"Chiều 4/11, phát biểu trước Quốc hội về những ...",https://vnexpress.net/tong-bi-thu-khong-de-ngu...,Chính trị,2025-11-04T17:01:00-07:00,vnexpress,Chính trị
3,Phó chánh án Nguyễn Quốc Đoàn làm Phó tổng Tha...,"Ngày 4/11, Thủ tướng bổ nhiệm ông Nguyễn Quốc ...",Ông Đoàn đảm nhiệm cương vị mới thay ông Nguyễ...,https://vnexpress.net/pho-chanh-an-nguyen-quoc...,Chính trị,2025-11-04T16:42:00-07:00,vnexpress,Chính trị
4,Ông Nguyễn Văn Quảng làm Bí thư Đảng ủy Tòa án...,"Ông Nguyễn Văn Quảng, Phó tổng thường trực Tha...","Chiều 4/11, Ban Thường vụ Đảng ủy các cơ quan ...",https://vnexpress.net/ong-nguyen-van-quang-lam...,Chính trị,2025-11-04T15:58:00-07:00,vnexpress,Chính trị


Hàm tách câu

In [4]:
import re

def sentence_split(text: str):
    if not isinstance(text, str):
        return []

    text = text.strip()
    if not text:
        return []

    sents = sent_tokenize(text)

    clean_sents = []
    for s in sents:
        # làm sạch khoảng trắng
        s = re.sub(r"\s+", " ", s).strip()
        if not s:
            continue

        # bỏ caption ảnh / video, câu quá ngắn
        if re.match(r"^(Ảnh|Photo|Video)\b", s, flags=re.IGNORECASE):
            continue
        if len(s.split()) < 5:
            continue

        clean_sents.append(s)

    return clean_sents

print("\nVí dụ tách câu (sentence_split):")
for i in range(min(3, len(df))):
    sents = sentence_split(df.loc[i, "body"])
    print(f"- Mẫu {i} có {len(sents)} câu, câu đầu:", sents[0] if sents else "")



Ví dụ tách câu (sentence_split):
- Mẫu 0 có 31 câu, câu đầu: Sáng 5/11, phát biểu khai mạc Hội nghị Trung ương 14, Tổng Bí thư Tô Lâm cho biết tại Hội nghị Trung ương 13, trên cơ sở đề nghị của Bộ Chính trị, Ban Chấp hành Trung ương khóa 13 đã biểu quyết thống nhất, tập trung cao, giới thiệu nhân sự Ban Chấp hành Trung ương khóa 14 - chưa bao gồm các Ủy viên Bộ Chính trị, Ủy viên Ban Bí thư đủ điều kiện tái cử và các trường hợp đặc biệt.
- Mẫu 1 có 4 câu, câu đầu: Tại phiên khai mạc, Tổng Bí thư Tô Lâm chủ trì và phát biểu định hướng thảo luận; Chủ tịch nước Lương Cường điều hành chương trình làm việc.
- Mẫu 2 có 48 câu, câu đầu: Chiều 4/11, phát biểu trước Quốc hội về những điểm mới và định hướng lớn trong dự thảo Văn kiện trình Đại hội 14, Tổng Bí thư cho rằng Quốc hội dành trọn một buổi để đại biểu thực hiện "trọng trách rất đặc biệt" - góp ý cho các văn kiện định hướng con đường phát triển đất nước trong nhiều năm tới.


Gom toàn bộ câu + TF-IDF

In [5]:
all_sentences = []
doc_sent_indices = []  

offset = 0
for idx, text in enumerate(df["body"].astype(str).tolist()):
    sents = sentence_split(text)
    if not sents:
        doc_sent_indices.append((idx, offset, offset))
        continue
    all_sentences.extend(sents)
    start = offset
    offset += len(sents)
    end = offset
    doc_sent_indices.append((idx, start, end))

print("\nTổng số câu trong toàn bộ corpus:", len(all_sentences))

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=5
)
tfidf_matrix = tfidf.fit_transform(all_sentences)
print("TF-IDF matrix shape:", tfidf_matrix.shape)



Tổng số câu trong toàn bộ corpus: 257771
TF-IDF matrix shape: (257771, 5000)


Hàm tạo short_summary

In [6]:
MAX_WORDS_SHORT = 220 
def build_short_summary_for_article(
    sentences,
    sent_emb_sparse,
    max_words=MAX_WORDS_SHORT,
    base_clusters=8,
    top_multi_clusters=2,
):
    """
    sentences: list[str]
    sent_emb_sparse: scipy.sparse matrix (n_sent, dim) từ TF-IDF
    """
    n_sent = len(sentences)
    if n_sent == 0:
        return ""

    # Bài rất ngắn → không cần KMeans
    if n_sent <= 3:
        out_sents = []
        cur_words = 0
        for s in sentences:
            n_w = len(s.split())
            if cur_words + n_w > max_words:
                break
            out_sents.append(s)
            cur_words += n_w
        return " ".join(out_sents).strip()

    # Số cụm động theo độ dài bài
    if n_sent <= 6:
        n_clusters = min(3, n_sent)
    elif n_sent <= 15:
        n_clusters = min(6, n_sent)
    elif n_sent <= 30:
        n_clusters = min(base_clusters, n_sent)
    else:
        n_clusters = min(base_clusters + 2, n_sent)  # bài rất dài → thêm cụm

    # Chuyển sang dense cho bài này
    sent_emb = sent_emb_sparse.toarray()

    # KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(sent_emb)
    centers = kmeans.cluster_centers_

    # Độ quan trọng cụm = kích thước cụm
    cluster_importance = []
    for c in range(n_clusters):
        idx_c = np.where(labels == c)[0]
        cluster_importance.append((c, len(idx_c)))
    cluster_importance.sort(key=lambda x: -x[1])  # giảm dần

    chosen_idx = []

    for rank, (c, _) in enumerate(cluster_importance):
        idx_c = np.where(labels == c)[0]
        if len(idx_c) == 0:
            continue

        center = centers[c]
        vecs = sent_emb[idx_c]
        norms = np.linalg.norm(vecs, axis=1) * np.linalg.norm(center) + 1e-8
        sims = np.dot(vecs, center) / norms  # cosine similarity

        order = np.argsort(-sims)   # giảm dần
        idx_ranked = idx_c[order]

        # Cụm quan trọng nhất → lấy 2 câu, cụm còn lại 1 câu
        if rank < top_multi_clusters and len(idx_ranked) >= 2:
            to_take = idx_ranked[:2]
        else:
            to_take = idx_ranked[:1]

        for idx_sent in to_take:
            if idx_sent not in chosen_idx:
                chosen_idx.append(idx_sent)

    # Sắp xếp theo thứ tự xuất hiện
    chosen_idx = sorted(chosen_idx)
    summary_sents = [sentences[i] for i in chosen_idx]

    # Ghép câu cho tới khi chạm max_words
    out_sents = []
    cur_words = 0
    for s in summary_sents:
        n_w = len(s.split())
        if cur_words + n_w > max_words:
            break
        out_sents.append(s)
        cur_words += n_w

    return " ".join(out_sents).strip()


Áp dụng: tạo short_summary cho toàn bộ dữ liệu

In [7]:
short_summaries = []

for (idx, start, end) in tqdm(doc_sent_indices, desc="Build short_summary"):
    if start == end:
        short_summaries.append("")
        continue

    sentences = all_sentences[start:end]
    sent_emb_sparse = tfidf_matrix[start:end]

    summary = build_short_summary_for_article(sentences, sent_emb_sparse)
    short_summaries.append(summary)

df["short_summary"] = short_summaries

df["short_len_words"] = df["short_summary"].str.split().str.len()
print("\nĐộ dài short_summary (words):")
print(df["short_len_words"].describe(percentiles=[0.5, 0.9, 0.95, 0.99]))

# Bỏ mẫu không có short_summary
before = len(df)
df = df[df["short_summary"].str.len() > 0].reset_index(drop=True)
print(f"\nĐã bỏ {before - len(df)} mẫu không có short_summary. Còn lại:", len(df))


Build short_summary: 100%|██████████| 11383/11383 [07:11<00:00, 26.40it/s]



Độ dài short_summary (words):
count    11383.000000
mean       195.445137
std         26.099063
min          0.000000
50%        202.000000
90%        217.000000
95%        219.000000
99%        220.000000
max        220.000000
Name: short_len_words, dtype: float64

Đã bỏ 30 mẫu không có short_summary. Còn lại: 11353


Xuất file summarize_data.csv cho training

In [8]:
from pathlib import Path

SUM_PATH = DATA_DIR / "summarize_data.csv"

# Tạo summarize_data.csv: input_text + target_text + label
summ_df = pd.DataFrame({
    "input_text": (
        df["title"].astype(str).str.strip() + ". " +
        df["body"].astype(str).str.strip()
    ).str.strip(),
    "target_text": df["short_summary"].astype(str).str.strip(),
})

# Thêm nhãn từ subject
if "subject" in df.columns:
    summ_df["label"] = df["subject"].astype(str).str.strip()
else:
    summ_df["label"] = ""

# Bỏ mẫu rỗng
before = len(summ_df)
summ_df = summ_df[
    (summ_df["input_text"].str.len() > 0) &
    (summ_df["target_text"].str.len() > 0) &
    (summ_df["label"].str.len() > 0)
].reset_index(drop=True)
print(f"Sau khi bỏ input/target/label rỗng: {before} -> {len(summ_df)} mẫu")

# Thống kê nhanh độ dài target_text
summ_df["target_len_words"] = summ_df["target_text"].str.split().str.len()
print("\nMô tả độ dài target_text (words):")
print(summ_df["target_len_words"].describe(percentiles=[0.5, 0.9, 0.95, 0.99]))
summ_df = summ_df.drop(columns=["target_len_words"])

DATA_DIR.mkdir(parents=True, exist_ok=True)
summ_df.to_csv(SUM_PATH, index=False, encoding="utf-8-sig")
print("\nĐã lưu summarize_data.csv:", SUM_PATH, "| shape:", summ_df.shape)
print("Các cột:", summ_df.columns.tolist())


Sau khi bỏ input/target/label rỗng: 11353 -> 11353 mẫu

Mô tả độ dài target_text (words):
count    11353.000000
mean       195.961596
std         24.119416
min         45.000000
50%        202.000000
90%        217.000000
95%        219.000000
99%        220.000000
max        220.000000
Name: target_len_words, dtype: float64

Đã lưu summarize_data.csv: d:\do-an-tot-nghiep\dataset\summarize_data.csv | shape: (11353, 3)
Các cột: ['input_text', 'target_text', 'label']
